In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
import unidecode

import folium
from folium import plugins

import geopy
from geopy.geocoders import Nominatim
#geolocator = Nominatim(user_agent="https://nominatim.openstreetmap.org/")
geolocator = Nominatim(user_agent="https://maps.googleapis.com/")

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

pd.options.plotting.backend = "plotly"
pd.set_option('max_columns', None)

The following data frame is a cleaned version of orginal data of COVID in Colombia, some of the clean done is:

* Formated headders upper 
* 'Fechas' formated in datetime
* Added 'Asintomático FIS' feature, extracted from 'FIS'
* Remove puntuation upper for categorical features
* Upper case for categorical features
* Fill NaN with 'No Info' in categorical features
* Drop rows where 'Estado'= ASINTOMATICO where 'atención'= HOSPITAL
* Drop rows where 'Estado'= ASINTOMATICO where 'atención'=HOSPITAL UCI
* Remove rows where 'FIS'<'Fecha diagnostico' does not meet.
* Remove rows where 'Fecha diagnostico'<'Fecha recuperado' does not meet.
* Remove rows where two values are contain in the same row for 'Fecha de mmuerte' and 'Fecha recueprado'
* Drop people above 99 year's old


Remarks:
* Checked that 'Asintomático FIS' match with 'Estado'=ASINTOMATICO
* Validated that we have not different values for atentción feature when Estado = 'FALLECIDO'
* We have people who were diagnosed with COVID after death
* We have missing values in 'Codigo pais'
* Checked that Estado=ASINTOMATICO have no Fecha de muerte, the people that died before diagnosed with COVID, seems that they not die becasueof the COVID
* Checked that we have no 'Tipo recuperación' values when 'Fecha de Muerte' is not null.

In [4]:
df=pd.read_pickle('pre_cleaned_dataset.pkl')

In [34]:
print(df.shape)
df.head(2)

(356068, 22)


,ID de caso,Fecha de notificación,Código DIVIPOLA,Ciudad de ubicación,Departamento o Distrito,atención,Edad,Sexo,Tipo,Estado,País de procedencia,FIS,Fecha de muerte,Fecha diagnostico,Fecha recuperado,fecha reporte web,Tipo recuperación,Codigo departamento,Codigo pais,Pertenencia etnica,Nombre grupo etnico,Asintomático FIS
0,1,2020-03-02,11001,BOGOTA D.C.,BOGOTA D.C.,RECUPERADO,19,F,IMPORTADO,LEVE,ITALIA,2020-02-27,NaT,2020-03-06,2020-03-13,2020-03-06,PCR,11,380.0,OTRO,NO INFO,NO INFO
1,2,2020-03-06,76111,GUADALAJARA DE BUGA,VALLE DEL CAUCA,RECUPERADO,34,M,IMPORTADO,LEVE,ESPANA,2020-03-04,NaT,2020-03-09,2020-03-19,2020-03-09,PCR,76,724.0,OTRO,NO INFO,NO INFO


In [32]:
#Re check in the previous notebook
# porque hay fechas de diagnostico sin Fecha, cómo saben si tiene COVID si nunca se les diagnostico, de donde sale encontes esa fehca de reporte web?
# hubo una fecha de inicio de sintomas FIS pero no hubo fecha de diagnostico, pero si reporte, es raro
t=df[~(df['Fecha diagnostico']<df['Fecha recuperado'])]
t[~t['Fecha recuperado'].isna()].select_dtypes('datetime')

,Fecha de notificación,FIS,Fecha de muerte,Fecha diagnostico,Fecha recuperado,fecha reporte web
11698,2020-05-04,NaT,NaT,NaT,2020-06-03,2020-05-12
24710,2020-04-26,2020-04-21,NaT,NaT,2020-05-21,2020-05-28
24711,2020-05-02,2020-05-02,NaT,NaT,2020-06-20,2020-05-28
24713,2020-04-01,2020-03-25,NaT,NaT,2020-06-20,2020-05-28
24714,2020-05-06,2020-05-03,NaT,NaT,2020-06-08,2020-05-28
...,...,...,...,...,...,...
206574,2020-07-20,2020-06-23,NaT,NaT,2020-07-23,2020-07-21
206575,2020-07-20,2020-06-24,NaT,NaT,2020-07-25,2020-07-21
220201,2020-06-24,2020-06-24,NaT,NaT,2020-07-25,2020-07-23
229565,2020-07-01,2020-06-23,NaT,NaT,2020-07-25,2020-07-24


Seems good for every feature, becasue NO INFO means there is not values (nan). And RECUPERADO must be the only one with valid data.